In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu_devices[0], True)

Using TensorFlow backend.


In [60]:
model=load_model('./models/new_model_fix_cat19.h5')

In [129]:
test = pd.read_csv('./data/test_data.csv',header=None)

In [131]:
test=test.iloc[:,0]

In [134]:
test

0                        [45, 97, 45, 56, 98, 45, 53, 99]
1       [92, 102, 114, 97, 99, 123, 50, 125, 123, 53, ...
2                  [55, 43, 51, 121, 45, 55, 43, 52, 121]
3       [45, 92, 108, 101, 102, 116, 40, 51, 94, 123, ...
4                                            [97, 62, 55]
                              ...                        
2995                 [45, 53, 97, 43, 55, 98, 45, 56, 99]
2996    [123, 45, 49, 125, 94, 123, 45, 54, 97, 43, 57...
2997                                     [97, 61, 45, 52]
2998                                     [97, 60, 45, 57]
2999                                         [97, 61, 55]
Name: 0, Length: 3000, dtype: object

In [133]:
num=0
for s in test:
        test[num] = bytearray(s, 'utf-8')
        num+=1

In [135]:
X_test = sequence.pad_sequences(test, maxlen=200)

In [137]:
score=model.predict(X_test)

In [142]:
score_max=100
for i in range(len(score)):
    if score_max>np.max(score[i]):
        score_max=np.max(score[i])

In [143]:
score_max

0.99116427

In [115]:
for i in range(len(model.predict(X_test))):
    print(np.argmax(model.predict(X_test)[i]))

17
17
2
17
17
2
1
1
1
10


In [4]:
data = pd.read_csv("./math_expr_new_9.csv",header=None)

In [84]:
def mexc_pre(math_expression):
    data = pd.read_csv("{}".format(math_expression),header=None)
       
    num=0
    for s in data.iloc[:,0]:
        data.iloc[num,0] = bytearray(s, 'utf-8')
        num+=1
    
    X_test = sequence.pad_sequences(data.iloc[:,0], maxlen=200)
    pre = model.predict(X_test)
    
    pre_list=[]
    for i in range(len(pre)) :
        if np.argmax(pre[i]) == 0 :
            pre_list.append('distributor_expression')
        elif np.argmax(pre[i]) == 1 :
            pre_list.append('linear_expression')
        elif np.argmax(pre[i]) == 2 :
            pre_list.append('linear_equation')
    print(pre_list)

In [14]:
def change_model(model_name) :
    global model
    model = load_model('{}'.format(model_name))

In [20]:
def model_fit(data_name,epoch,save_name):
    data = pd.read_csv("{}".format(data_name),header=None)
    data.columns=['Math Expresion','Expression Name']
    data.loc[data["Expression Name"]=='distributor_expression','Expression Name']=0
    data.loc[data["Expression Name"]=='linear_expression','Expression Name']=1
    data.loc[data["Expression Name"]=='linear_equation','Expression Name']=2
    
    num=0
    for s in data.iloc[:,0]:
        data.iloc[num,0] = bytearray(s, 'utf-8')
        num+=1
    
    earlystop = EarlyStopping(monitor='val_loss', mode='min',min_delta=0.0001, verbose = 1, patience=5)
    mc = ModelCheckpoint('./models/model_{}.h5'.format(save_name), monitor='val_loss', mode='min', save_best_only=True)
    
    X_train,X_test,y_train,y_test = train_test_split(data.iloc[:,0],data.iloc[:,1],test_size=0.2)
    X_train_seq = sequence.pad_sequences(X_train, maxlen=200)
    X_test_seq = sequence.pad_sequences(X_test, maxlen=200)
    Y_train_ctg = np_utils.to_categorical(y_train)
    Y_test_ctg = np_utils.to_categorical(y_test)
    
    model = Sequential()
    model.add(Embedding(len(X_train), 100))
    model.add(LSTM(100, activation='tanh'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    history = model.fit(X_train_seq, Y_train_ctg, batch_size=128, 
                    epochs=epoch,validation_data=(X_test_seq, Y_test_ctg),callbacks=[earlystop,mc],
                       verbose=2)

In [21]:
model_fit('math_expr_new_20.csv',200)

Train on 16 samples, validate on 4 samples
Epoch 1/200
 - 0s - loss: 1.0986 - accuracy: 0.1875 - val_loss: 1.0935 - val_accuracy: 0.5000
Epoch 2/200
 - 0s - loss: 1.0941 - accuracy: 0.5000 - val_loss: 1.0881 - val_accuracy: 0.5000
Epoch 3/200
 - 0s - loss: 1.0893 - accuracy: 0.5000 - val_loss: 1.0821 - val_accuracy: 0.5000
Epoch 4/200
 - 0s - loss: 1.0837 - accuracy: 0.5000 - val_loss: 1.0751 - val_accuracy: 0.5000
Epoch 5/200
 - 0s - loss: 1.0770 - accuracy: 0.5000 - val_loss: 1.0669 - val_accuracy: 0.5000
Epoch 6/200
 - 0s - loss: 1.0687 - accuracy: 0.5000 - val_loss: 1.0570 - val_accuracy: 0.5000
Epoch 7/200
 - 0s - loss: 1.0581 - accuracy: 0.5000 - val_loss: 1.0448 - val_accuracy: 0.5000
Epoch 8/200
 - 0s - loss: 1.0441 - accuracy: 0.5000 - val_loss: 1.0293 - val_accuracy: 0.5000
Epoch 9/200
 - 0s - loss: 1.0251 - accuracy: 0.5000 - val_loss: 1.0084 - val_accuracy: 0.5000
Epoch 10/200
 - 0s - loss: 0.9985 - accuracy: 0.5625 - val_loss: 0.9786 - val_accuracy: 0.5000
Epoch 11/200
 - 

In [22]:
change_model('./models/model_math_expr_new_20.h5')

In [23]:
mexc_pre('./new_samples.csv')

['linear_equation', 'linear_equation', 'distributor_expression', 'distributor_expression', 'distributor_expression', 'linear_equation', 'distributor_expression', 'linear_equation', 'distributor_expression', 'linear_equation', 'linear_expression', 'linear_expression', 'linear_equation', 'distributor_expression', 'linear_equation', 'linear_equation', 'linear_equation', 'distributor_expression', 'linear_equation', 'linear_equation', 'linear_equation', 'linear_equation', 'distributor_expression', 'distributor_expression', 'linear_expression', 'linear_equation', 'linear_equation', 'linear_equation', 'distributor_expression', 'linear_equation', 'linear_expression', 'linear_expression', 'distributor_expression', 'linear_equation', 'distributor_expression', 'linear_equation', 'distributor_expression', 'linear_expression', 'linear_equation', 'linear_equation', 'linear_equation', 'distributor_expression', 'distributor_expression', 'linear_equation', 'distributor_expression', 'distributor_express

In [11]:
mexc_pre('./new_samples.csv')

['linear_equation', 'distributor_expression', 'distributor_expression', 'linear_equation', 'distributor_expression', 'linear_equation', 'linear_equation', 'linear_equation', 'linear_equation', 'linear_equation', 'linear_expression', 'linear_expression', 'linear_equation', 'linear_equation', 'linear_expression', 'distributor_expression', 'distributor_expression', 'linear_equation', 'linear_expression', 'linear_equation', 'linear_expression', 'distributor_expression', 'linear_equation', 'linear_equation', 'linear_expression', 'linear_equation', 'linear_equation', 'linear_expression', 'distributor_expression', 'linear_expression', 'linear_expression', 'linear_expression', 'linear_equation', 'linear_equation', 'linear_equation', 'linear_expression', 'distributor_expression', 'linear_expression', 'linear_expression', 'linear_equation', 'linear_expression', 'distributor_expression', 'distributor_expression', 'distributor_expression', 'distributor_expression', 'distributor_expression', 'distr